# EBM Feature/Term Importances

In this notebook we show how to compute and interpret Overall Importances shown in InterpretML's Global Explanations for EBMs. We also show how to compute importances of a group of features or terms.

Throughout the notebook we use _term_ to denote both single features and interactions (pairs).

## Train an Explainable Boosting Machine (EBM) for a regression task

Let's use the Boston dataset as a reference and train an EBM.

In [ ]:
import pandas as pd
from sklearn.datasets import load_boston
from interpret.glassbox import ExplainableBoostingRegressor

boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df["target"] = boston.target

train_cols = df.columns[0:-1]
label = df.columns[-1]
X = df[train_cols]
y = df[label]

ebm = ExplainableBoostingRegressor(random_state=1)
ebm.fit(X, y) 

## Explain the Model

EBMs provide two different kinds of explanations: global explanations about the overall model behavior and local explanations about individual predictions from the model.

### Global Explanation

Global Explanations are useful for understanding what a model finds important, as well as identifying potential flaws in its decision making or the training data. Let's start by computing and displaying a global explanation:

In [ ]:
from interpret import show

ebm_global = ebm.explain_global(name='EBM')
show(ebm_global)

The overall importance for each term is calculated as _the average absolute contribution (score) a term (feature or pair) makes when predicting across the training dataset._ This way of measuring term importance tends to favor terms which, on average, have large impact on predictions for many cases. The overall importance is not a measure of positive/negative -- it is a measure of how important each term is in the scores. For regression, these scores are represented in the same units as the y-axis of the feature graphs. For classification, the scores would be in logits.

Going beyond overall term importances, because EBMs are additive models we can measure exactly how each term contributes to a prediction. Let's take a look at the graph of the term, `LSTAT`, by selecting it in the drop-down menu.

![Global Explanation - LSTAT](../assets/importance_notebook_global_lstat.png)

The way to interpret this is that if a new datapoint came in with `LSTAT` = 5, the model adds about +2.7 to the final prediction. However, for a different datapoint with `LSTAT` = 10, the model would now add approx. -0.47 to the prediction.

To make individual predictions, the model uses each term graph as a look up table, notes the contribution per term, and sums them together with the learned intercept to make a prediction. In regression, the intercept is the mean target (label) of the training set, and each term adds or subtracts to this mean. In classification, the intercept reflects the base rate of the positive class on a log scale. The gray above and below the graph shows the confidence of the model in that region of the graph.

### Local Explanations

We can see the full breakdown of a prediction on a single sample with Local Explanations. Here's how to compute the prediction breakdown for the first sample in our dataset:

In [ ]:
from interpret import show
index = 0
show(ebm.explain_local(X[index:index+1], y[index:index+1]))

Let's take a look at the prediction by selecting it in the drop-down menu.

![Local Explanation](../assets/importance_notebook_local_exp.png)

The model prediction is 26.8. We can see that the intercept adds about +22.5, `LSTAT` adds ~+2.7, and `RAD` adds about -1.2. So far, for the top 3 contributing terms, we're at a cumulative prediction of ~+24. If we repeat this process for all the terms, we'll arrive exactly at the model prediction of 26.8.

## Viewing _all_ term importances

Due to space limitations in our graphs, the term importance summary only shows the top 15 terms. To view the overall importances of all terms of a trained EBM - the scores shown in the global explanation summary - we use `term_importances()`:

In [ ]:
importances = ebm.term_importances()
names = ebm.term_names_

for (term_name, importance) in zip(names, importances):
    print(f"Term {term_name} importance: {importance}")

Note that mean absolute contribution isn't the only way of calculating term importances. Another metric our package provides is the `min_max` option, which computes the difference between the `max` (the highest score on the graph) and `min` (the lowest score on the graph) values for each term. Term importance measured with `min_max` is a measure of the maximum impact a term can have, even though it might have this amount of impact on very few cases, whereas `avg_weight`(the default parameter) is a measure of typical (average) contribution of a term across all cases.

In [ ]:
importances = ebm.term_importances("min_max")
names = ebm.term_names_

for (term, importance) in zip(names, importances):
    print(f"Term {term} importance: {importance}")

## Feature/Term Group Importances

We provide utility functions to compute the importances of groups of features or terms and, optionally, append these importances to the global feature attribution bar graph. Note that shape function graphs are not generated for groups of features/terms, just their overall importance is shown on the Summary.

Grouping terms and then calculating and displaying their importance does not change the model and the predictions it makes in any way -- group importances are just a method for computing the importance of groups of terms in addition to the importances of individual terms that are already calculated. As you'll see in the examples below, it's OK for features/terms to overlap in different groups.

### Computing group importances

Let's use the Adult dataset and train an EBM for a classification task.

In [ ]:
import pandas as pd
from interpret.glassbox import ExplainableBoostingClassifier

df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    header=None)
df.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]

train_cols = df.columns[0:-1]
label = df.columns[-1]

X = df[train_cols]
y = df[label].apply(lambda x: 0 if x == " <=50K" else 1)

adult_ebm = ExplainableBoostingClassifier(random_state=1)
adult_ebm.fit(X, y)

We then create a list of terms -- single features or interactions -- as our group and compute its importance:

In [ ]:
from interpret.glassbox.ebm.research.group_importance import *

social_feature_group = ["MaritalStatus", "Relationship", "Race", "Gender", "NativeCountry"]
importance = compute_group_importance(social_feature_group, adult_ebm, X)
print(f"Group: {social_feature_group} - Importance: {importance}")

In this example we create a group with five terms and compute its importance. Similar to single feature importances, we interpret this score as _the average absolute contribution this group of terms makes when predicting across the training dataset._ Note that for each prediction, the contribution of each term in the group will be added before taking the absolute value. 

We also have the option to create a global explanation containing the group importance or append it to an existing explanation:

In [ ]:
my_global_exp = append_group_importance(social_feature_group, adult_ebm, X)
show(my_global_exp)

The importance of `social_feature_group` is about 1.30, which is higher than the importance of any individual feature/term:

![Global Explanation - Social Feature Group](../assets/importance_notebook_social_group.png)

We could make this type of comparison between different groups too:

In [ ]:
education_feature_group = ["Education", "EducationNum"]
relationship_feature_group = ["MaritalStatus", "Relationship"]
social_feature_group = ["MaritalStatus", "Relationship", "Race", "Gender", "NativeCountry"]
my_global_exp = append_group_importance(social_feature_group, adult_ebm, X)
my_global_exp = append_group_importance(education_feature_group, adult_ebm, X, global_exp=my_global_exp)
my_global_exp = append_group_importance(relationship_feature_group, adult_ebm, X, global_exp=my_global_exp)
show(my_global_exp)

The importance of `education_feature_group` is about 0.52, higher than each of its individual terms but smaller than some individual terms such as `Age`. Remember, creating groups of features/terms does not, in any way, change the model and its predictions, it only allows you to estimate the importance of these groups.

This graph, for example, suggests that features related to relationships are more important than features reated to education.

![Global Explanation - Education Group](../assets/importance_notebook_education_group.png)

We can also compare a group we are interested in (e.g. `social_feature_group`) with a group of all other reamining terms.

In [ ]:
social_feature_group = ["MaritalStatus", "Relationship", "Race", "Gender", "NativeCountry"]
all_other_terms = [term for term in adult_ebm.term_names_ if term not in social_feature_group]

my_global_exp = append_group_importance(social_feature_group, adult_ebm, X)
my_global_exp = append_group_importance(all_other_terms, adult_ebm, X, group_name="all_other_terms", global_exp=my_global_exp)
show(my_global_exp)

Note that `all_other_terms` has the highest importance score, followed by  `social_feature_group`. 

![Global Explanation - All Other Group](../assets/importance_notebook_all_other_group.png)

It's even possible to create a group with all terms.

In [ ]:
all_terms_group = [term for term in adult_ebm.term_names_]
mew_global_exp = append_group_importance(all_terms_group, adult_ebm, X, group_name="all_terms")
show(mew_global_exp)

Finally, we also expose a function to compute the importances of a group of terms as well as all the model's original terms.

In [ ]:
my_dict = get_group_and_individual_importances([social_feature_group, education_feature_group], adult_ebm, X)
for key in my_dict:
    print(f"Term: {key} - Importance: {my_dict[key]}")